# 作为客户端与HTTP服务交互

发送一个简单的`HTTP GET`请求到远程的服务上

In [4]:
from urllib import request, parse

# Base URL being accessed --- get --- 
url = 'http://httpbin.org/get'

# Dictionary of query parameters (if any)
parms = {
   'name1' : 'value1',
   'name2' : 'value2'
}

# Encode the query string
querystring = parse.urlencode(parms)

# Make a GET request and read the response
u = request.urlopen(url+'?' + querystring)
resp = u.read()
resp

b'{\n  "args": {\n    "name1": "value1", \n    "name2": "value2"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.6"\n  }, \n  "origin": "101.13.44.99, 101.13.44.99", \n  "url": "https://httpbin.org/get?name1=value1&name2=value2"\n}\n'

如果你需要使用`POST`方法在请求主体中发送查询参数，可以将参数编码后作为可选参数提供给 `urlopen()` 函数

In [6]:
from urllib import request, parse

# Base URL being accessed   --- post ---
url = 'http://httpbin.org/post'

# Dictionary of query parameters (if any)
parms = {
   'name1' : 'value1',
   'name2' : 'value2'
}

# Encode the query string --- encode('ascii') ---
querystring = parse.urlencode(parms).encode('ascii')

# Make a POST request and read the response
u = request.urlopen(url, querystring)
resp = u.read()

如果你需要在发出的请求中提供一些自定义的`HTTP`头，例如修改 `user-agent` 字段,可以创建一个包含字段值的字典，并创建一个`Request`实例然后将其传给 `urlopen()`

In [5]:
from urllib import request, parse

# Base URL being accessed   --- post ---
url = 'http://httpbin.org/post'

# Dictionary of query parameters (if any)
parms = {
   'name1' : 'value1',
   'name2' : 'value2'
}

# Encode the query string --- encode('ascii') ---
querystring = parse.urlencode(parms).encode('ascii')


# Extra headers
headers = {
    'User-agent' : 'none/ofyourbusiness',
    'Spam' : 'Eggs'
}

req = request.Request(url, querystring, headers=headers)

# Make a request and read the response
u = request.urlopen(req)
resp = u.read()

## requests 库

关于`requests`库，一个值得一提的特性就是它能以多种方式从请求中返回响应结果的内容。从上面的代码来看，`resp.text` 带给我们的是以`Unicode`解码的响应文本。但是，如果去访问 `resp.content` ，就会得到原始的二进制数据。另一方面，如果访问 `resp.json` ，那么就会得到`JSON`格式的响应内容

In [10]:
import requests

# Base URL being accessed
url = 'http://httpbin.org/post'

# Dictionary of query parameters (if any)
parms = {
   'name1' : 'value1',
   'name2' : 'value2'
}

# Extra headers
headers = {
    'User-agent' : 'none/ofyourbusiness',
    'Spam' : 'Eggs'
}

resp = requests.post(url, data=parms, headers=headers)

# Decoded text returned by the request
text = resp.text
print(text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name1": "value1", 
    "name2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "25", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "Spam": "Eggs", 
    "User-Agent": "none/ofyourbusiness"
  }, 
  "json": null, 
  "origin": "101.13.44.99, 101.13.44.99", 
  "url": "https://httpbin.org/post"
}



下面这个示例利用 `requests` 库发起一个`HEAD`请求，并从响应中提取出一些`HTTP`头数据的字段

In [6]:
import requests

resp = requests.head('http://www.python.org/index.html')

status = resp.status_code
#last_modified = resp.headers['last-modified'] # KeyError: 'last-modified'
#content_type = resp.headers['content-type'] # KeyError: 'content-type'
content_length = resp.headers['content-length']

content_length

'0'

利用`requests`通过基本认证登录`Pypi`的例子

In [10]:
import requests

resp = requests.get('http://pypi.python.org/pypi?:action=login',
                    auth=('user','password'))
resp

<Response [200]>

利用`requests`将`HTTP cookies`从一个请求传递到另一个的例子

In [12]:
import requests

url = 'http://httpbin.org/post'

# First request
resp1 = requests.get(url)
...

# Second requests with cookies received on first requests
resp2 = requests.get(url, cookies=resp1.cookies)

Ellipsis

In [16]:
import requests
url = 'http://httpbin.org/post'
files = { 'file': ('file.txt', open('file.txt', 'rb')) }

r = requests.post(url, files=files)
r.text

'{\n  "args": {}, \n  "data": "", \n  "files": {\n    "file": "Hello, world."\n  }, \n  "form": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "157", \n    "Content-Type": "multipart/form-data; boundary=4eaed52ef10a49aa87a6cce31a8b26f7", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.21.0"\n  }, \n  "json": null, \n  "origin": "101.13.44.99, 101.13.44.99", \n  "url": "https://httpbin.org/post"\n}\n'

## 讨论

在开发过程中测试 `HTTP` 客户端代码常常是很令人沮丧的，因为所有棘手的细节问题都需要考虑（例如 `cookies` 、认证、 `HTTP` 头、编码方式等）。要完成这些任务，考虑使用[ `httpbin` 服务](http://httpbin.org)。这个站点会接收发出的请求，然后以`JSON`的形式将相应信息回传回来。下面是一个交互式的例子：

In [24]:
import requests

r = requests.get(
    'http://httpbin.org/get?name=Dave&n=37',
    headers = { 'User-agent': 'goaway/1.0' }
)
resp = r.json()
resp.keys()
resp['headers']
resp['args']

dict_keys(['args', 'headers', 'origin', 'url'])

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Host': 'httpbin.org',
 'User-Agent': 'goaway/1.0'}

{'n': '37', 'name': 'Dave'}

`request` 库还对许多高级的`HTTP`客户端协议提供了支持，比如`OAuth`。 [`requests` 模块的文档](http://docs.python-requests.org)质量很高

# 创建TCP服务器

## [socket] 菜鳥教程

http://www.runoob.com/python/python-socket.html

```py
# server.py

import socket

# 建立一个服务端
server = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
#绑定要监听的端口
server.bind(('localhost',6999))
#开始监听 表示可以使用五个链接排队
server.listen(5) 
while True:
    # conn就是客户端链接过来而在服务端为期生成的一个链接实例
    # 等待链接,多个链接的时候就会出现问题,其实返回了两个值
    conn,addr = server.accept() 
    print(conn,addr)
    while True:
        try:
            # 接收数据
            data = conn.recv(1024)  
            # 打印接收到的数据
            print('recive:',data.decode()) 
            # 然后再发送数据
            conn.send(data.upper()) 
        except ConnectionResetError as e:
            print('关闭了正在占线的链接！')
            break
    conn.close()
```

```py
# client.py

import socket

# 客户端 发送一个数据，再接收一个数据
# 声明socket类型，同时生成链接对象
client = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
# 建立一个链接，连接到本地的6969端口
client.connect(('localhost',6999)) 
while True:
    # addr = client.accept()
    # print('连接地址：', addr)
    msg = '欢迎访问菜鸟教程！'  
    # 发送一条信息 python3 只接收btye流
    client.send(msg.encode('utf-8'))  
    # 接收一个信息，并指定接收的大小 为1024字节
    data = client.recv(1024) 
    # 输出我接收的信息
    print('recv:',data.decode()) 
# 关闭这个链接
client.close() 
```



## [socketserver]创建TCP服务器


```py
# TCP.py
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        while True:

            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)

if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler)
    serv.serve_forever()
```    

为了测试这个服务器，运行它并打开另外一个`Python`进程连接这个服务器：

```py
>>> from socket import socket, AF_INET, SOCK_STREAM
>>> s = socket(AF_INET, SOCK_STREAM)
>>> s.connect(('localhost', 20000))
>>> s.send(b'Hello')
5
>>> s.recv(8192)
b'Hello'
>>>
```

很多时候，可以很容易的定义一个不同的处理器。下面是一个使用 `StreamRequestHandler` 基类将一个类文件接口放置在底层`socket`上的例子

```py
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        # self.rfile is a file-like object for reading
        for line in self.rfile:
            # self.wfile is a file-like object for writing
            self.wfile.write(line)

if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler)
    serv.serve_forever()
```


## 讨论

`socketserver` 可以让我们很容易的创建简单的`TCP`服务器。 但是，你需要注意的是，默认情况下这种服务器是单线程的，一次只能为一个客户端连接服务。 如果你想处理多个客户端，可以初始化一个 `ForkingTCPServer` 或者是 `ThreadingTCPServer` 对象

```py
from socketserver import ThreadingTCPServer


if __name__ == '__main__':
    serv = ThreadingTCPServer(('', 20000), EchoHandler)
    serv.serve_forever()
```

使用`fork`或线程服务器有个潜在问题就是它们会为每个客户端连接创建一个新的进程或线程。 由于客户端连接数是没有限制的，因此一个恶意的黑客可以同时发送大量的连接让你的服务器奔溃。

如果你担心这个问题，你可以创建一个 **预先分配大小** 的工作线程池或进程池。 你先创建一个普通的非线程服务器，然后在一个线程池中使用 `serve_forever()` 方法来启动它们。

```py
if __name__ == '__main__':
    from threading import Thread
    NWORKERS = 16
    serv = TCPServer(('', 20000), EchoHandler)
    for n in range(NWORKERS):
        t = Thread(target=serv.serve_forever)
        t.daemon = True
        t.start()
    serv.serve_forever()
```

一般来讲，一个 `TCPServer` 在实例化的时候会绑定并激活相应的 `socket` 。 不过，有时候你想通过设置某些选项去调整底下的 `socket` ，可以设置参数 `bind_and_activate=False` 。如下：

```py
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler, bind_and_activate=False)
    # Set up various socket options
    serv.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)
    # Bind and activate
    serv.server_bind()
    serv.server_activate()
    serv.serve_forever()
```

上面的 `socket` 选项是一个非常普遍的配置项，它允许服务器重新绑定一个之前使用过的端口号。 由于要被经常使用到，它被放置到类变量中，可以直接在 `TCPServer` 上面设置。 在实例化服务器的时候去设置它的值，如下所示：

```py
if __name__ == '__main__':
    TCPServer.allow_reuse_address = True
    serv = TCPServer(('', 20000), EchoHandler)
    serv.serve_forever()
```

在上面示例中，我们演示了两种不同的处理器基类（ `BaseRequestHandler` 和 `StreamRequestHandler` ）。 `StreamRequestHandler` 更加灵活点，能通过设置其他的类变量来支持一些新的特性。比如：


```py
import socket

class EchoHandler(StreamRequestHandler):
    # Optional settings (defaults shown)
    timeout = 5                      # Timeout on all socket operations
    rbufsize = -1                    # Read buffer size
    wbufsize = 0                     # Write buffer size
    disable_nagle_algorithm = False  # Sets TCP_NODELAY socket option
    def handle(self):
        print('Got connection from', self.client_address)
        try:
            for line in self.rfile:
                # self.wfile is a file-like object for writing
                self.wfile.write(line)
        except socket.timeout:
            print('Timed out!')
```

部分`Python`的高层网络模块（比如`HTTP`、`XML-RPC`等）都是建立在 `socketserver` 功能之上。 也就是说，直接使用 `socket` 库来实现服务器也并不是很难。 下面是一个使用 `socket` 直接编程实现的一个服务器简单例子：

```py
from socket import socket, AF_INET, SOCK_STREAM

def echo_handler(address, client_sock):
    print('Got connection from {}'.format(address))
    while True:
        msg = client_sock.recv(8192)
        if not msg:
            break
        client_sock.sendall(msg)
    client_sock.close()

def echo_server(address, backlog=5):
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(address)
    sock.listen(backlog)
    while True:
        client_sock, client_addr = sock.accept()
        echo_handler(client_addr, client_sock)

if __name__ == '__main__':
    echo_server(('', 20000))
```









# 创建UDP服务器

跟`TCP`一样，`UDP`服务器也可以通过使用 `socketserver` 库很容易的被创建。 例如，下面是一个简单的时间服务器：

```py
# UDP.py

from socketserver import BaseRequestHandler, UDPServer
import time

class TimeHandler(BaseRequestHandler):
    def handle(self):
        print('Got connection from', self.client_address)
        # Get message and client socket
        msg, sock = self.request
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), self.client_address)

if __name__ == '__main__':
    serv = UDPServer(('', 20000), TimeHandler)
    serv.serve_forever()

```

你先定义一个实现 `handle()` 特殊方法的类，为客户端连接服务。 这个类的 `request` 属性是一个包含了数据报和底层 `socket` 对象的元组。 `client_address` 包含了客户端地址

打开另外一个`Python`进程向服务器发送消息：

```py
>>> from socket import socket, AF_INET, SOCK_DGRAM
>>> s = socket(AF_INET, SOCK_DGRAM)
>>> s.sendto(b'', ('localhost', 20000))
0
>>> s.recvfrom(8192)
(b'Wed Aug 15 20:35:08 2012', ('127.0.0.1', 20000))
>>>
```

## 讨论

一个典型的`UDP`服务器接收到达的数据报(消息)和客户端地址。如果服务器需要做应答， 它要给客户端回发一个数据报。对于数据报的传送， 你应该使用`socket`的 `sendto()` 和 `recvfrom()` 方法。 尽管传统的 `send()` 和 `recv()` 也可以达到同样的效果， 但是前面的两个方法对于`UDP`连接而言更普遍。

由于没有底层的连接，`UPD`服务器相对于`TCP`服务器来讲实现起来更加简单。 不过，`UDP`天生是不可靠的（因为通信没有建立连接，消息可能丢失）。 因此需要由你自己来决定该怎样处理丢失消息的情况

不过通常来说，如果可靠性对于你程序很重要，你需要借助于序列号、重试、超时以及一些其他方法来保证。

 `UDP`通常被用在那些对于可靠传输要求不是很高的场合。例如，在实时应用如多媒体流以及游戏领域， 无需返回恢复丢失的数据包（程序只需简单的忽略它并继续向前运行）

`UDPServer` 类是单线程的，也就是说一次只能为一个客户端连接服务。 实际使用中，这个无论是对于`UDP`还是`TCP`都不是什么大问题。 如果你想要并发操作，可以实例化一个 `ForkingUDPServer` 或 `ThreadingUDPServer` 对象：

```py
from socketserver import ThreadingUDPServer

   if __name__ == '__main__':
    serv = ThreadingUDPServer(('',20000), TimeHandler)
    serv.serve_forever()
```

直接使用 `socket` 来实现一个`UDP`服务器也不难，下面是一个例子：

```py
from socket import socket, AF_INET, SOCK_DGRAM
import time

def time_server(address):
    sock = socket(AF_INET, SOCK_DGRAM)
    sock.bind(address)
    while True:
        msg, addr = sock.recvfrom(8192)
        print('Got message from', addr)
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), addr)

if __name__ == '__main__':
    time_server(('', 20000))
```






# 通过CIDR地址生成对应的IP地址集

In [24]:
import ipaddress
net = ipaddress.ip_network('123.45.67.64/31')

# '123.45.67.64/32' => 123.45.67.64
# '123.45.67.64/31' => 64 65
# '123.45.67.64/30' => 64~67[4筆]
# ~'123.45.67.64/26' => 64~127[64筆]

net
for a in net:
    print(a)

IPv4Network('123.45.67.64/31')

123.45.67.64
123.45.67.65


In [36]:

# '12:3456:78:90ab:cd:ef01:23:30/128' => 12:3456:78:90ab:cd:ef01:23:30
# '12:3456:78:90ab:cd:ef01:23:30/127' => 30 31
# ~'12:3456:78:90ab:cd:ef01:23:30/124' => 30~39 3a~3f[共16筆]

net6 = ipaddress.ip_network('12:3456:78:90ab:cd:ef01:23:30/127')
for a in net6:
    print(a)

12:3456:78:90ab:cd:ef01:23:30
12:3456:78:90ab:cd:ef01:23:31


`Network` 也允许像`List`一样的索引取值 成员检查

In [38]:
net.num_addresses
net[0]
net[-1]

a = ipaddress.ip_address('123.45.67.65')
a in net

2

IPv4Address('123.45.67.64')

IPv4Address('123.45.67.65')

True

## 讨论

`ipaddress` 模块有很多类可以表示`IP`地址、网络和接口。 当你需要操作网络地址（比如解析、打印、验证等)

要注意的是， `ipaddress` 模块跟其他一些和网络相关的模块比如 `socket` 库交集很少。 所以，你不能使用 `IPv4Address` 的实例来代替一个地址字符串，你首先得显式的使用 `str()` 转换它

```py
a = ipaddress.ip_address('127.0.0.1')
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)

#s.connect((a, 8080))
#TypeError: Can't convert 'IPv4Address' object to str implicitly

s.connect((str(a), 8080))

```

# [cgi]创建一个简单的REST接口{會使用Django框架建構;cgi未接觸}

构建一个`REST`风格的接口最简单的方法是创建一个基于`WSGI`标准（`PEP 3333`）的很小的库，下面是一个例子：

```py
# resty.py

import cgi

def notfound_404(environ, start_response):
    start_response('404 Not Found', [ ('Content-type', 'text/plain') ])
    return [b'Not Found']

class PathDispatcher:
    def __init__(self):
        self.pathmap = { }

    def __call__(self, environ, start_response):
        path = environ['PATH_INFO']
        params = cgi.FieldStorage(environ['wsgi.input'],
                                  environ=environ)
        method = environ['REQUEST_METHOD'].lower()
        environ['params'] = { key: params.getvalue(key) for key in params }
        handler = self.pathmap.get((method,path), notfound_404)
        return handler(environ, start_response)

    def register(self, method, path, function):
        self.pathmap[method.lower(), path] = function
        return function
```

为了使用这个调度器，你只需要编写不同的处理器，就像下面这样：

```py
import time

_hello_resp = '''\
<html>
  <head>
     <title>Hello {name}</title>
   </head>
   <body>
     <h1>Hello {name}!</h1>
   </body>
</html>'''

def hello_world(environ, start_response):
    start_response('200 OK', [ ('Content-type','text/html')])
    params = environ['params']
    resp = _hello_resp.format(name=params.get('name'))
    yield resp.encode('utf-8')

_localtime_resp = '''\
<?xml version="1.0"?>
<time>
  <year>{t.tm_year}</year>
  <month>{t.tm_mon}</month>
  <day>{t.tm_mday}</day>
  <hour>{t.tm_hour}</hour>
  <minute>{t.tm_min}</minute>
  <second>{t.tm_sec}</second>
</time>'''

def localtime(environ, start_response):
    start_response('200 OK', [ ('Content-type', 'application/xml') ])
    resp = _localtime_resp.format(t=time.localtime())
    yield resp.encode('utf-8')

if __name__ == '__main__':
    from resty import PathDispatcher
    from wsgiref.simple_server import make_server

    # Create the dispatcher and register functions
    dispatcher = PathDispatcher()
    dispatcher.register('GET', '/hello', hello_world)
    dispatcher.register('GET', '/localtime', localtime)

    # Launch a basic server
    httpd = make_server('', 8080, dispatcher)
    print('Serving on port 8080...')
    httpd.serve_forever()
```

要测试下这个服务器，你可以使用一个浏览器或 `urllib` 和它交互。例如：

```py
>>> u = urlopen('http://localhost:8080/hello?name=Guido')
>>> print(u.read().decode('utf-8'))
<html>
  <head>
     <title>Hello Guido</title>
   </head>
   <body>
     <h1>Hello Guido!</h1>
   </body>
</html>

>>> u = urlopen('http://localhost:8080/localtime')
>>> print(u.read().decode('utf-8'))
<?xml version="1.0"?>
<time>
  <year>2012</year>
  <month>11</month>
  <day>24</day>
  <hour>14</hour>
  <minute>49</minute>
  <second>17</second>
</time>
>>>
```

`PathDispatcher` 类。 这个分发器仅仅只是管理一个字典，将(方法,路径)对映射到处理器函数上面。 

当一个请求到来时，它的方法和路径被提取出来，然后被分发到对应的处理器上面去。 

另外，任何查询变量会被解析后放到一个字典中，以 `environ['params']` 形式存储。 后面这个步骤太常见，所以建议你在分发器里面完成，这样可以省掉很多重复代码。

使用分发器的时候，你只需简单的创建一个实例，然后通过它注册各种WSGI形式的函数。 编写这些函数应该超级简单了，只要你遵循 `start_response()` 函数的编写规则，并且最后返回字节字符串 `byte` 即可




## 讨论[REST API 介紹]

在编写`REST`接口时，通常都是服务于普通的`HTTP`请求。但是跟那些功能完整的网站相比，你通常只需要处理数据。 这些数据以各种标准格式编码，比如 `XML` 、 `JSON` 或 `CSV`

长期运行的程序可能会使用一个`REST API`来实现监控或诊断。 大数据应用程序可以使用`REST`来构建一个数据查询或提取系统

`REST API`已经被大量客户端编程环境所支持，比如`Javascript`, `Android`, `iOS`等。 因此，利用这种接口可以让你开发出更加复杂的应用程序


### WSGI标准[cgi 為例]

为了实现一个简单的REST接口，你只需让你的程序代码满足Python的WSGI标准即可。 WSGI被标准库支持，同时也被绝大部分第三方web框架支持

```py
import cgi

def wsgi_app(environ, start_response):
    pass
```
`environ` 属性是一个字典，包含了从`web`服务器如`Apache`(参考 `Internet RFC 3875` )提供的`CGI`接口中获取的值。 要将这些不同的值提取出来，你可以像这么这样写：

```py
def wsgi_app(environ, start_response):
    # environ['REQUEST_METHOD'] 代表请求类型如GET、POST、HEAD等
    method = environ['REQUEST_METHOD']
    # environ['PATH_INFO'] 表示被请求资源的路径
    path = environ['PATH_INFO']
    
    # cgi.FieldStorage() 可以从请求中提取查询参数
    # 并将它们放入一个类字典对象中以便后面使用
    # Parse the query parameters
    params = cgi.FieldStorage(environ['wsgi.input'], environ=environ)
```

`start_response` 参数是一个为了初始化一个请求对象而必须被调用的函数。 第一个参数是返回的`HTTP`状态值，第二个参数是一个(名,值)元组列表，用来构建返回的`HTTP`头。例如：

```py
def wsgi_app(environ, start_response):
    pass
    start_response('200 OK', [('Content-type', 'text/plain')])
```    

为了返回数据，一个`WSGI`程序必须返回一个字节字符串序列。可以像下面这样使用一个列表来完成：

```py
def wsgi_app(environ, start_response):
    pass
    start_response('200 OK', [('Content-type', 'text/plain')])
    resp = []
    resp.append(b'Hello World\n')
    resp.append(b'Goodbye!\n')
    return resp
```

或者，你还可以使用 `yield` ：

```py
def wsgi_app(environ, start_response):
    pass
    start_response('200 OK', [('Content-type', 'text/plain')])
    yield b'Hello World\n'
    yield b'Goodbye!\n'
```
    
这里要强调的一点是最后返回的必须是 `byte`

尽管`WSGI`程序通常被定义成一个函数，不过你也可以使用类实例来实现，只要它实现了合适的 `__call__()` 方法。例如：

```py
class WSGIApplication:
    def __init__(self):
        ...
    def __call__(self, environ, start_response)
       ...
```




# 通过XML-RPC实现简单的远程调用

实现一个远程方法调用的最简单方式是使用`XML-RPC`。下面我们演示一下一个实现了键-值存储功能的简单服务器：

```py
from xmlrpc.server import SimpleXMLRPCServer

class KeyValueServer:
    _rpc_methods_ = ['get', 'set', 'delete', 'exists', 'keys']
    def __init__(self, address):
        self._data = {}
        self._serv = SimpleXMLRPCServer(address, allow_none=True)
        for name in self._rpc_methods_:
            self._serv.register_function(getattr(self, name))

    def get(self, name):
        return self._data[name]

    def set(self, name, value):
        self._data[name] = value

    def delete(self, name):
        del self._data[name]

    def exists(self, name):
        return name in self._data

    def keys(self):
        return list(self._data)

    def serve_forever(self):
        self._serv.serve_forever()

# Example
if __name__ == '__main__':
    kvserv = KeyValueServer(('', 15000))
    kvserv.serve_forever()
```

下面我们从一个客户端机器上面来访问服务器：

```py
>>> from xmlrpc.client import ServerProxy
>>> s = ServerProxy('http://localhost:15000', allow_none=True)
>>> s.set('foo', 'bar')
>>> s.set('spam', [1, 2, 3])
>>> s.keys()
['spam', 'foo']
>>> s.get('foo')
'bar'
>>> s.get('spam')
[1, 2, 3]
>>> s.delete('spam')
>>> s.exists('spam')
False
>>>
```




## 讨论

`XML-RPC` 可以让我们很容易的构造一个简单的远程调用服务。你所需要做的仅仅是创建一个服务器实例， 通过它的方法 `register_function()` 来注册函数，然后使用方法 `serve_forever()` 启动它。 在上面我们将这些步骤放在一起写到一个类中，不够这并不是必须的。比如你还可以像下面这样创建一个服务器：

```py
from xmlrpc.server import SimpleXMLRPCServer
def add(x,y):
    return x+y

serv = SimpleXMLRPCServer(('', 15000))
serv.register_function(add)
serv.serve_forever()

```

一般来讲，你不应该将 `XML-RPC` 服务以公共`API`的方式暴露出来。 对于这种情况，通常分布式应用程序会是一个更好的选择。

`XML-RP`C的一个缺点是它的性能

# [multiprocessing.connection]在不同的Python解释器之间交互

通过使用 `multiprocessing.connection` 模块可以很容易的实现解释器之间的通信。 下面是一个简单的应答服务器例子：

```py
from multiprocessing.connection import Listener
import traceback

def echo_client(conn):
    try:
        while True:
            msg = conn.recv()
            conn.send(msg)
    except EOFError:
        print('Connection closed')

def echo_server(address, authkey):
    serv = Listener(address, authkey=authkey)
    while True:
        try:
            client = serv.accept()

            echo_client(client)
        except Exception:
            traceback.print_exc()

echo_server(('', 25000), authkey=b'peekaboo')

```

然后客户端连接服务器并发送消息的简单示例：

```py
>>> from multiprocessing.connection import Client
>>> c = Client(('localhost', 25000), authkey=b'peekaboo')
>>> c.send('hello')
>>> c.recv()
'hello'
>>> c.send(42)
>>> c.recv()
42
>>> c.send([1, 2, 3, 4, 5])
>>> c.recv()
[1, 2, 3, 4, 5]
>>>
```

跟底层 `socket` 不同的是，每个消息会完整保存（每一个通过 `send()` 发送的对象能通过 `recv()` 来完整接受）。 另外，所有对象会通过 `pickle` 序列化。因此，任何兼容 `pickle` 的对象都能在此连接上面被发送和接受

## 讨论[簡易消息通信]

目前有很多用来实现各种消息传输的包和函数库，比如`ZeroMQ`、`Celery`等。 
你还有另外一种选择就是自己在底层`socket`基础之上来实现一个消息传输层。 

但是你想要简单一点的方案，那么这时候 `multiprocessing.connection` 就派上用场了。 仅仅使用一些简单的语句即可实现多个解释器之间的消息通信。

**UNIX域套接字**

```py
s = Listener('/tmp/myconn', authkey=b'peekaboo')
```

**Windows命名管道**

```py
s = Listener(r'\\.\pipe\myconn', authkey=b'peekaboo')

```

一个通用准则是，你不要使用 `multiprocessing` 来实现一个对外的公共服务。 `Client()` 和 `Listener()` 中的 `authkey` 参数用来认证发起连接的终端用户。 如果密钥不对会产生一个异常。

此外，该模块最适合用来建立长连接（而不是大量的短连接）， 例如，两个解释器之间启动后就开始建立连接并在处理某个问题过程中会一直保持连接状态。

如果你需要对底层连接做更多的控制，比如需要支持超时、非阻塞`I/O`或其他类似的特性， 你最好使用另外的库或者是在高层`socket`上来实现这些特性。

# 实现远程方法调用

你想在一个消息传输层如 `sockets` 、`multiprocessing connections` 或 `ZeroMQ` 的基础之上实现一个简单的远程过程调用（ `RPC` ）

**解决方案**

将函数请求、参数和返回值使用 `pickle` 编码后，在不同的解释器直接传送 `pickle` 字节字符串，可以很容易的实现 `RPC` 。 下面是一个简单的 `PRC` 处理器，可以被整合到一个服务器中去：

要注意的是很多消息层（比如 `multiprocessing` ）已经使用 `pickle` 序列化了数据。 如果是这样的话，对 `pickle.dumps()` 和 `pickle.loads()` 的调用要去掉

In [14]:
%%file rpcserver.py

import pickle


class RPCHandler:
    def __init__(self):
        self._functions = { }

    def register_function(self, func):
        self._functions[func.__name__] = func

    def handle_connection(self, connection):
        try:
            while True:
                # Receive a message
                func_name, args, kwargs = pickle.loads(connection.recv())
                # Run the RPC and send a response
                try:
                    r = self._functions[func_name](*args,**kwargs)
                    connection.send(pickle.dumps(r))
                except Exception as e:
                    connection.send(pickle.dumps(e))
        except EOFError:
             pass
            
            
# 要使用这个处理器，你需要将它加入到一个消息服务器中。
# 你有很多种选择，但是使用 multiprocessing 库是最简单的。
# 下面是一个RPC服务器例子：            
            
            
from multiprocessing.connection import Listener
from threading import Thread

def rpc_server(handler, address, authkey):
    sock = Listener(address, authkey=authkey)
    while True:
        client = sock.accept()
        t = Thread(target=handler.handle_connection, args=(client,))
        t.daemon = True
        t.start()

# Some remote functions
def add(x, y):
    return x + y

def sub(x, y):
    return x - y

# Register with a handler
handler = RPCHandler()
handler.register_function(add)
handler.register_function(sub)

# Run the server
print('Run the server')
rpc_server(handler, ('localhost', 17000), authkey=b'peekaboo')

Overwriting rpcserver.py


In [15]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'rpcserver.py')
cmd = "python " + f'"{DIR}"'
print(cmd,BASE_DIR,sep='\n')
#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\rpcserver.py"
D:\CODE\GitHub\py\資料結構\py3-cookbook


为了从一个远程客户端访问服务器，你需要创建一个对应的用来传送请求的`RPC`代理类

In [6]:
import pickle

class RPCProxy:
    def __init__(self, connection):
        self._connection = connection
    def __getattr__(self, name):
        def do_rpc(*args, **kwargs):
            self._connection.send(pickle.dumps((name, args, kwargs)))
            result = pickle.loads(self._connection.recv())
            if isinstance(result, Exception):
                raise result
            return result
        return do_rpc

要使用这个代理类，你需要将其包装到一个服务器的连接上面

In [12]:
from multiprocessing.connection import Client
c = Client(('localhost', 17000), authkey=b'peekaboo')
proxy = RPCProxy(c)
proxy.add(2, 3)

proxy.sub(2, 3)

5

-1

In [13]:
proxy.sub([1, 2], 4)

TypeError: unsupported operand type(s) for -: 'list' and 'int'